In [1]:
import os
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
path_driver = 'C:/Windows/chromedriver.exe'
url = 'https://overwatchleague.com/en-us/stats'
dirname = os.path.dirname(os.getcwd())
# dirname

In [3]:
# path_save = os.path.relpath('overwatchleague-stats.csv')
# path_save = 'C:/Users/aelhabr/Documents/projects/overwatch/data-raw/overwatchleague-stats.csv'
path_save

'C:/Users/aelhabr/Documents/projects/overwatch/data-raw/overwatchleague-stats.csv'

In [4]:
opts = webdriver.chrome.options.Options()
opts.set_headless()
# assert opts.headless
driver = webdriver.Chrome(path_driver, options=opts)

In [5]:
driver.get(url)
time.sleep(1)

In [6]:
num_pg = 1
file_data = []

In [7]:
# References: 
# + https://codereview.stackexchange.com/questions/87901/copy-table-using-selenium-and-python
# + https://stackoverflow.com/questions/24166689/selenium-python-access-next-pages-of-search-results

while True:
    try:
        link = driver.find_element_by_link_text(str(num_pg))
    except NoSuchElementException:
        with open(path_save, 'w') as f:
            f.write('\n'.join(file_data))
        print('done')
        break
        
    link.click()
    # print(driver.current_url)
    table = driver.find_element_by_class_name('Table')
    head = table.find_element_by_tag_name('thead')
    body = table.find_element_by_tag_name('tbody')
    
    if num_pg == 1:
        file_header = []
        head_line = head.find_element_by_tag_name('tr')
        headers = head_line.find_elements_by_tag_name('th')

        for header in headers:
            file_header.append(header.text)
        file_data.append(",".join(file_header))
    
    body_rows = body.find_elements_by_tag_name('tr')
    for row in body_rows:
        data = row.find_elements_by_tag_name('td')
        file_row = []
        
        # This is a hack to get the tooltip for the role.
        role = data[0].get_attribute('title')
        for datum in data:
            if datum.text == '':
                datum_text = role
            else:
                datum_text = datum.text
            file_row.append(datum_text.lower().replace(',', ''))
        file_data.append(','.join(file_row))
    
    num_pg += 1

done
